## Oblique linearly polarized Alfvén wave

References:

- @vasquezMakingAlfvenicFluctuation1996

> We now consider hybrid simulations (see Vasquez [1995] for details of the code) of the evolution of an initially linearly polarized . Alfven wave train with A = 0.3 and traveling at OB = 60°. We choose a small wavenumher of 0.025(c/w¢) -1 where c/wm is an ion inertial length. At this wavenumber ion wave dispersion is negligible over the total simulation time. In the simulation box, 4 cycles of this wave will be included. Time in the simulation is normalized to an ion gyrocycle ft. In units of (c/c~)f~, the Alfvdn speed is 1, and the speed of the wave train is cos 60° = 0.5. The wave period is 502 ft-1. Electrons are treated as an isothermal fluid with temperature Te. Ions are assigned a Maxwellian velocity distribution with the bulk speed of an Alfvdn wave and temperature Ti equal to 2",. The total plasma /3 ks taken to be 0.275. We are especially concerned here with small/3 since this condition characterizes re~ons near the Sun. The isothermal sound speed corresponding to this/3 is 0.37, and this value will be used for c, to evaluate Equations (8)-(9). Evaluated in this way, the correspondence with the simulations is close but not exact because ions in a hybrid simulation do not obey a predetermined equation of state.

Notes:

- There seems to have an initialization problem for 1D simulation.

TODO:

- $\gamma$ in `warpx_momentum_spread_expressions`

In [1]:
import numpy as np
from pywarpx import picmi
from space_analysis.utils.math import cosd, sind
from space_analysis.simulation.warpx import HybridSimulation
import os
from pathlib import Path

In [2]:
constants = picmi.constants

In [3]:
def init_obl_alfven(
    k,
    B0,
    A,  #: relative amplitude
    theta,
    vA,
    n0,
    v_ti, #: ion thermal velocity
):
    """
    Generate field and distribution for an oblique linearly polarized Alfven wave.
    
    Note: field with a wave propagating along the z axis at a large angle `theta` with respect to the background magnetic field lying in the x-z plane.

    The initial waveis an Alfven mode in which the magnetic field fluctuation points along the y and z axis and has a relative amplitude $A = \delta B_y / B_0$
    """

    B0k = B0 * cosd(theta)
    B0k1 = B0 * sind(theta)

    Bz_expression = f"{B0k}"
    By_expression = f"{A * B0} * cos({k} * z)"
    Bx_expression = f"{B0k1}"
    pz_expression = 0
    px_expression = 0
    py_expression = f"{A * vA * cosd(theta)} * cos({k} * z)"
        
    field = picmi.AnalyticInitialField(
        Bx_expression=Bx_expression,
        By_expression=By_expression,
        Bz_expression=Bz_expression,
    )
    
    momentum_expressions = [px_expression, py_expression, pz_expression]
    rms_velocity = [v_ti, v_ti, v_ti]
    dist = picmi.AnalyticDistribution(
        density_expression=n0,
        momentum_expressions=momentum_expressions,
        warpx_momentum_spread_expressions=rms_velocity,
    )

    return field, dist

In [4]:
class AlfvenModes(HybridSimulation):
    # Applied field parameters
    dim: int = 1
    B0: float = 100 * 1e-9
    """Initial magnetic field strength (T)"""
    n0: float = 100 * 1e6
    """Initial plasma density (m^-3)"""

    A: float = 0.5  # relative amplitude
    theta: float = 0  # angle with respect to the background magnetic field
    wave_number: int = 4  # wave number

    # Spatial domain
    Lz_norm: float = 256
    Lx_norm: float = 16 # blocking_factor is 8 so at 16
    Ly_norm: float = 16 

    def setup_init_cond(self):
        """setup initial conditions"""
        self.k = 2 * np.pi / (self.Lz / self.wave_number)  # angular wavenumber

        B_ext, dist = init_obl_alfven(
            k=self.k,
            B0=self.B0,
            A=self.A,
            theta=self.theta,
            vA=self.vA,
            n0=self.n0,
            v_ti=self.v_ti,
        )
        self._B_ext = B_ext
        self._dist = dist

        return self

In [5]:
dim = 1
dim = 2
dim = 3
beta = 0.25
theta = 60

base_dir = Path(os.getcwd())
sub_dir = f"dim_{dim}_beta_{beta}_theta_{theta}"
directory = base_dir / sub_dir

In [6]:
os.makedirs(directory, exist_ok=True)
os.chdir(directory)

simulation = AlfvenModes(
    dim=dim,
    beta=beta,
    theta=theta,
    substeps=32,
    nppc=256,
    dz_norm=2,
    dt_norm=0.125,
    diag_part=True,
    plasma_resistivity=0,
)

Numerical parameters:
	dt = 1.8e-02 s
	total steps = 800

Numerical parameters (Hybrid):
	total simulation time = 100.0 (t_ci)
	time step = 0.125 (t_ci)
	CFL (with substeps) condition = 0.02

Initializing simulation with input parameters:
	Te = 61.931 eV
	n = 1.0e+02 cm^-3
	B0 = 100.00 nT
	M/m = 400

Plasma parameters:
	d_i = 1.1e+04 m
	t_ci = 1.4e-01 s
	v_ti = 1.7e+05 m/s
	vA = 4.7e+05 m/s
	vA/c = 0.0015568891097689005



In [7]:
simulation.dt_norm

0.00390625

In [10]:
# wavenumber
print("wavenumer (1 / ion inertial length):", simulation.k * simulation.d_i)

# wave period
w = simulation.k * simulation.vA
t_w = 2 * np.pi / w
t_w_norm = t_w / simulation.t_ci
t_w_norm

wavenumer (1 / ion inertial length): 0.09817477042468103


10.236719248384558

In [ ]:
simulation._sim.step()